## Example notebook for accessing data in a Swift object store

In [ ]:
import os, sys
import getpass

from os import listdir
from os.path import isfile, join

utils_dir = join(os.getcwd(), 'utils')
sys.path.append(utils_dir)
import SwiftStorageUtils

### Specify OpenStack credentials and container information

In [ ]:
# OpenStack credentials
os_username = 'hluetc'
os_tenant_name = 'helmchen.hifo.uzh'
os_auth_url = 'https://cloud.s3it.uzh.ch:5000/v2.0'
# provide OS password
os_password = getpass.getpass()

# store connection settings for OpenStack in dictionary
conn_opts = {
    "os_username": os_username,
    "os_password": os_password,
    "os_tenant_name": os_tenant_name,
    "os_auth_url": os_auth_url
}

In [ ]:
# Swift container (must exist already; create in https://cloud.s3it.uzh.ch/project/containers/)
swift_container = 'henry2'
# Pseudo-folder for data hierarchy
swift_folder = 'test'

### List items in container

In [ ]:
# check if container exists and print items
from SwiftStorageUtils import listItems
listItems(swift_container, conn_opts)

### Upload files to container

In [ ]:
# Define files that should be uploaded to Swift
# All files of type 'ftype' in the directory 'source_dir' will be transfered
source_dir = '/Users/Henry/polybox/Data_temp/NeuroPipeline/LEC_Data/'
ftype = '.h5' # set to None to choose all files, '.h5' to select only HDF5 files etc.

In [ ]:
# function to restrict file type (returns true for files ending with ftype)
def is_type(f, ftype):
    if not ftype:
        return True
    else:
        return (
                f.lower().endswith(ftype)
            )

In [ ]:
# assemble file list
files= [f for f in listdir(source_dir) 
            if (isfile(join(source_dir, f)) 
            and is_type(join(source_dir, f), ftype))]
files = [join(source_dir, f) for f in files]
print "Uploading %1.0f files to Swift:" % len(files)
for f in files: print(f)

In [ ]:
# upload items in files
from SwiftStorageUtils import uploadItems
uploadItems(swift_container, swift_folder, source_dir, files, conn_opts)

### Download files from container

In [ ]:
# specify the objects to be downloaded
objects_to_download = [
    'test/Monyer_Leitner_F400_spot02.h5',
    'test/Monyer_Leitner_F400_spot04.h5'
]
# local storage directory
local_dir = '/Users/Henry/Data/Projects/NeuroPipeline/temp'

# download options
down_opts = {
    'skip_identical': True,
    'out_directory': local_dir,
}

from SwiftStorageUtils import downloadItems
downloadItems(swift_container, objects_to_download, conn_opts, down_opts)

### Delete files in container

In [ ]:
# specify the objects to be deleted
objects_to_delete = [
    'test/Monyer_Leitner_F397_spot01.h5',
    'test/Monyer_Leitner_F296_spot01.h5'
]

from SwiftStorageUtils import deleteItems
deleteItems(swift_container, objects_to_delete, conn_opts)

In [ ]:
print file_list